In [83]:
import pandas as pd 
import numpy as np 

# getting NamUs Missing Persons database in 3 parts 
db1_missing = pd.read_csv("../data/missing-namus-first-pull.csv")
db2_missing = pd.read_csv("../data/missing-namus-second-pull.csv") 
db3_missing = pd.read_csv("../data/missing-namus-third-pull.csv") 

# combining all 3 into one dataframe 
missing_df_1 = pd.concat([db1_missing, db2_missing, db3_missing], ignore_index=True, sort=False)

# final db 
final_db = pd.DataFrame()

missing_df_1.head()

,Case Number,DLC,Legal Last Name,Legal First Name,Missing Age,City,County,State,Biological Sex,Race / Ethnicity,Date Modified
0,MP128712,09/04/2024,Campirano,Nixon,16 Years,Goodyear,Maricopa,AZ,Male,White / Caucasian,09/06/2024
1,MP128606,08/31/2024,Batra,Anjali,18 Years,Morgan Hill,Santa Clara,CA,Female,Other,09/04/2024
2,MP128466,08/30/2024,CLARK,JOSHUA,35 Years,Mulberry,Polk,FL,Male,White / Caucasian,09/06/2024
3,MP128454,08/30/2024,Turney-Long,Alinda,16 Years,Winter Haven,Polk,FL,Female,White / Caucasian,09/06/2024
4,MP128473,08/30/2024,COTE,KATHY,78 Years,West Palm Beach,Palm Beach,FL,Female,White / Caucasian,09/06/2024


In [84]:
missing_df_2 = missing_df_1.drop(["Case Number", "Legal Last Name", "Legal First Name", "Date Modified"], axis=1)

missing_df_2.head()

,DLC,Missing Age,City,County,State,Biological Sex,Race / Ethnicity
0,09/04/2024,16 Years,Goodyear,Maricopa,AZ,Male,White / Caucasian
1,08/31/2024,18 Years,Morgan Hill,Santa Clara,CA,Female,Other
2,08/30/2024,35 Years,Mulberry,Polk,FL,Male,White / Caucasian
3,08/30/2024,16 Years,Winter Haven,Polk,FL,Female,White / Caucasian
4,08/30/2024,78 Years,West Palm Beach,Palm Beach,FL,Female,White / Caucasian


In [ ]:
### DLC 

In [85]:
dlc_1 = pd.DataFrame(missing_df_2["DLC"])
dlc_1.head()

,DLC
0,09/04/2024
1,08/31/2024
2,08/30/2024
3,08/30/2024
4,08/30/2024


In [86]:
dlc_1.isnull().sum()

DLC    0
dtype: int64

In [87]:
dlc_1.describe()

,DLC
count,25060
unique,12159
top,01/01/2001
freq,17


In [88]:
dlc_1["DLC"] = dlc_1["DLC"].apply(lambda x : (x.split("/")[0] + "/" + x.split("/")[2]))
dlc_1.head()

,DLC
0,09/2024
1,08/2024
2,08/2024
3,08/2024
4,08/2024


In [89]:
from datetime import date

today = date.today()
currentMY = str(today).split("-")[1] + "/" + str(today).split("-")[0]

print(currentMY)

10/2024


In [90]:
def dateToNumerical(caseDate, todayDate): 

    caseMonth = caseDate.split("/")[0]
    caseYear = caseDate.split("/")[1]

    todayMonth = todayDate.split("/")[0]
    todayYear = todayDate.split("/")[1]

    if (caseYear == todayYear): 
        return ((int(todayMonth) - int(caseMonth)) / 12)
    
    else: 
        total = int(todayYear) - int(caseYear)

        if caseMonth >= todayMonth: 
            total = total - 1 
            total = total + ((int(todayMonth) + (12 - int(caseMonth))) / 12)

        else: 
            total = total + ((int(todayMonth) - int(caseMonth)) / 12)

        return total 


In [91]:
dlc_1["DLC"] = dlc_1["DLC"].apply(lambda x : dateToNumerical(x, currentMY))

dlc_1.head()

,DLC
0,0.083333
1,0.166667
2,0.166667
3,0.166667
4,0.166667


In [ ]:
# Normalize and Standardize 

In [92]:
dlc_1.describe()

,DLC
count,25060.000000
mean,18.605829
std,15.561851
min,0.083333
25%,5.083333
50%,14.833333
75%,29.833333
max,122.750000


In [93]:
dlc_1.boxplot(column='DLC')

<Axes: xlabel='Biological Sex'>

In [94]:
from sklearn import preprocessing

# Normalize 
scaler = preprocessing.MinMaxScaler()
d = scaler.fit_transform(dlc_1)
norm_stan_dlc_data = pd.DataFrame(d, columns=dlc_1.columns)

# Standardize 
norm_stan_dlc_data['DLC'] = (norm_stan_dlc_data['DLC'] - norm_stan_dlc_data['DLC'].mean()) / norm_stan_dlc_data['DLC'].std() 

norm_stan_dlc_data

,DLC
0,-1.190250
1,-1.184895
2,-1.184895
3,-1.184895
4,-1.184895
...,...
25055,3.843427
25056,3.864847
25057,4.207565
25058,4.582414


In [95]:
norm_stan_dlc_data.boxplot(column="DLC")

<Axes: xlabel='Biological Sex'>

In [96]:
final_db["Last Seen"] = norm_stan_dlc_data

In [97]:
final_db.head()

,Last Seen
0,-1.190250
1,-1.184895
2,-1.184895
3,-1.184895
4,-1.184895


In [98]:
## Age 
age_1 = pd.DataFrame(missing_df_2["Missing Age"])
age_1.head()

,Missing Age
0,16 Years
1,18 Years
2,35 Years
3,16 Years
4,78 Years


In [99]:
age_1.describe()

,Missing Age
count,25055
unique,136
top,16 Years
freq,973


In [100]:
# purpose: make every age value into a numerical value 
def ageToNumerical(df): 
    for idx, i in enumerate(df["Missing Age"]): 
        # for null/empty/nan values
        if pd.isna(i) or i == "" or isinstance(i, (int, float)) or pd.isnull(i): 
            df.loc[idx, "Missing Age"] = 34 # mean age of dataset 

    # handling special cases 
        # 1 - 10 age 
        elif str(i).strip() == '< 1 Year' or str(i).strip() == '1 - 2 Years': 
            df.loc[idx, "Missing Age"] = 1

        elif str(i).strip() == '3 - 4 Years': 
            df.loc[idx, "Missing Age"] = 3

        # 10 - 20 age 
        elif str(i).strip() == '14 - 15 Years': 
            df.loc[idx, "Missing Age"] = 14

        elif str(i).strip() == '15 - 17 Years': 
            df.loc[idx, "Missing Age"] = 16

        elif str(i).strip() == '16 - 18 Years' or str(i).strip() == '16 - 17 Years': 
            df.loc[idx, "Missing Age"] = 17

        elif str(i).strip() == '18 - 20 Years': 
            df.loc[idx, "Missing Age"] = 19


        # 20 - 30 age
        elif str(i).strip() == '21 - 22 Years': 
            df.loc[idx, "Missing Age"] = 21

        elif str(i).strip() == '23 - 24 Years' or str(i).strip() == '23 - 25 Years': 
            df.loc[idx, "Missing Age"] = 23

        elif str(i).strip() == '29 - 31 Years' or str(i).strip() == '25 - 35 Years' or str(i).strip() == '25 - 39 Years': 
            df.loc[idx, "Missing Age"] = 30

        # 30 - 40 age 
        elif str(i).strip() == '30 - 40 Years' or str(i).strip() == '35 - 37 Years' or str(i).strip() == '36 - 37 Years' or str(i).strip() == '33 - 34 Years' or str(i).strip() == '34 - 35 Years': 
            df.loc[idx, "Missing Age"] = 35

        elif str(i).strip() == '37 - 38 Years' or str(i).strip() == '30 - 31 Years': 
            df.loc[idx, "Missing Age"] = 37

        elif str(i).strip() == '39 - 41 Years' or str(i).strip() == '40 - 41 Years': 
            df.loc[idx, "Missing Age"] = 40

        # 40 - 50 age 
        elif str(i).strip() == '42 - 44 Years' or str(i).strip() == '40 - 47 Years' or str(i).strip() == '40 - 42 Years' or str(i).strip() == '44 - 45 Years' or str(i).strip() == '43 - 44 Years': 
            df.loc[idx, "Missing Age"] = 43

        elif str(i).strip() == '49 - 50 Years': 
            df.loc[idx, "Missing Age"] = 50

        # 50 - 60 age 
        elif str(i).strip() == '50 - 60 Years' or str(i).strip() == '51 - 52 Years': 
            df.loc[idx, "Missing Age"] = 55

        elif str(i).strip() == '55 - 65 Years' or str(i).strip() == '59 - 60 Years': 
            df.loc[idx, "Missing Age"] = 60

        elif str(i).strip() == '56 - 60 Years': 
            df.loc[idx, "Missing Age"] = 58

        # senior 
        elif str(i).strip() == '65 - 71 Years' or str(i).strip() == '80 - 82 Years': 
            df.loc[idx, "Missing Age"] = 70

    # most common cases
        # ex. "20 Years" 
        else: 
            df.loc[idx, "Missing Age"] = i[0:-5]

    # string to float 
    df["Missing Age"] = df["Missing Age"].astype(float)

    return df 
    

age_2 = ageToNumerical(age_1) 

In [101]:
age_2.head()

,Missing Age
0,16.0
1,18.0
2,35.0
3,16.0
4,78.0


In [102]:
age_2.describe()

,Missing Age
count,25060.000000
mean,34.406464
std,17.278007
min,1.000000
25%,21.000000
50%,32.000000
75%,45.000000
max,116.000000


In [103]:
age_2.isnull().sum()

Missing Age    0
dtype: int64

In [ ]:
# Normalize and Standardize 

In [104]:
age_2.boxplot(column="Missing Age")

<Axes: xlabel='Biological Sex'>

In [105]:
# Normalize 
scaler = preprocessing.MinMaxScaler()
d = scaler.fit_transform(age_2)
norm_stan_age_data = pd.DataFrame(d, columns=age_2.columns)

# Standardize 
norm_stan_age_data['Missing Age'] = (norm_stan_age_data['Missing Age'] - norm_stan_age_data['Missing Age'].mean()) / norm_stan_age_data['Missing Age'].std() 

norm_stan_age_data

,Missing Age
0,-1.065312
1,-0.949558
2,0.034352
3,-1.065312
4,2.523065
...,...
25055,-0.486541
25056,-0.602295
25057,-0.544418
25058,-0.833804


In [106]:
norm_stan_age_data.boxplot(column="Missing Age")

<Axes: xlabel='Biological Sex'>

In [107]:
final_db["Age"] = norm_stan_age_data 

In [108]:
final_db.head()

,Last Seen,Age
0,-1.190250,-1.065312
1,-1.184895,-0.949558
2,-1.184895,0.034352
3,-1.184895,-1.065312
4,-1.184895,2.523065


In [ ]:
### Gender 

In [109]:
gender_1 = pd.DataFrame(missing_df_2["Biological Sex"])
gender_1.head()

,Biological Sex
0,Male
1,Female
2,Male
3,Female
4,Female


In [110]:
gender_1.isnull().sum()

Biological Sex    0
dtype: int64

In [111]:
gender_1.describe()

,Biological Sex
count,25060
unique,3
top,Male
freq,15705


In [112]:
gender_1.value_counts().plot(kind="bar")

<Axes: xlabel='Biological Sex'>

In [113]:
gender_1.value_counts()

Biological Sex
Male              15705
Female             9349
Other                 6
Name: count, dtype: int64

In [ ]:
# Hot encoding 

In [114]:
gender_2 = pd.get_dummies(gender_1['Biological Sex'], prefix='Gender')

In [115]:
gender_2.head()

,Gender_Female,Gender_Male,Gender_Other
0,False,True,False
1,True,False,False
2,False,True,False
3,True,False,False
4,True,False,False


In [116]:
final_db = pd.concat([final_db, gender_2], axis=1)
# df = pd.concat([scaled_con_data, categorical], axis=1)

In [117]:
final_db.head()

,Last Seen,Age,Gender_Female,Gender_Male,Gender_Other
0,-1.190250,-1.065312,False,True,False
1,-1.184895,-0.949558,True,False,False
2,-1.184895,0.034352,False,True,False
3,-1.184895,-1.065312,True,False,False
4,-1.184895,2.523065,True,False,False


In [ ]:
### Race 

In [118]:
race_1 = pd.DataFrame(missing_df_2["Race / Ethnicity"]) 
race_1.head()

,Race / Ethnicity
0,White / Caucasian
1,Other
2,White / Caucasian
3,White / Caucasian
4,White / Caucasian


In [119]:
race_1.describe()

,Race / Ethnicity
count,25060
unique,75
top,White / Caucasian
freq,14072


In [120]:
# purpose: reduce Race/Ethnicity categories 
def reduceRaceCategories(df): 
    # new column for new race categories
    df["Race Category"] = ""

    for idx, i in enumerate(df["Race / Ethnicity"]): 
        # to new column for race category 
        if i == "Hispanic / Latino": 
            df.loc[idx, "Race Category"] = "Hispanic / Latino" 

        elif i == "White / Caucasian": 
            df.loc[idx, "Race Category"] = "White / Caucasian" 
            
        elif i == "Black / African American": 
            df.loc[idx, "Race Category"] = "Black / African American" 
            
        elif i == "American Indian / Alaska Native": 
            df.loc[idx, "Race Category"] = "American Indian / Alaska Native" 
            
        elif i == "Asian": 
            df.loc[idx, "Race Category"] = "Asian"
        # for mixed races / other
        else: 
            df.loc[idx, "Race Category"] = "Mix / Other / Unknown"

    df = df.drop(["Race / Ethnicity"], axis=1)

    return df 

race_2 = reduceRaceCategories(race_1) 

In [121]:
for idx, i in enumerate(race_2["Race Category"]): 
    if pd.isna(i) or str(i).strip() == "": 
        race_2.loc[idx, "Race Category"] = "Mix / Other / Unknown" 
        print("Empty")

race_2["Race Category"] = race_2["Race Category"].replace(r'^\s*$', np.nan, regex=True)
race_2["Race Category"].fillna("Mix / Other / Unknown", inplace=True)

C:\Users\Jason Rubio\AppData\Local\Temp\ipykernel_5764\1582913077.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  race_2["Race Category"].fillna("Mix / Other / Unknown", inplace=True)


In [122]:
race_2.value_counts()

Race Category                  
White / Caucasian                  14072
Black / African American            4087
Hispanic / Latino                   3466
Mix / Other / Unknown               2061
American Indian / Alaska Native      797
Asian                                577
Name: count, dtype: int64

In [123]:
race_2.describe()

,Race Category
count,25060
unique,6
top,White / Caucasian
freq,14072


In [ ]:
# Hot encoding 

In [124]:
race_3 = pd.get_dummies(race_2["Race Category"])
race_3.head()

,American Indian / Alaska Native,Asian,Black / African American,Hispanic / Latino,Mix / Other / Unknown,White / Caucasian
0,False,False,False,False,False,True
1,False,False,False,False,True,False
2,False,False,False,False,False,True
3,False,False,False,False,False,True
4,False,False,False,False,False,True


In [125]:
final_db = pd.concat([final_db, race_3], axis=1)

In [126]:
final_db.head()

,Last Seen,Age,Gender_Female,Gender_Male,Gender_Other,American Indian / Alaska Native,Asian,Black / African American,Hispanic / Latino,Mix / Other / Unknown,White / Caucasian
0,-1.190250,-1.065312,False,True,False,False,False,False,False,False,True
1,-1.184895,-0.949558,True,False,False,False,False,False,False,True,False
2,-1.184895,0.034352,False,True,False,False,False,False,False,False,True
3,-1.184895,-1.065312,True,False,False,False,False,False,False,False,True
4,-1.184895,2.523065,True,False,False,False,False,False,False,False,True


In [ ]:
### County and City and State 

In [127]:
county = missing_df_2["County"] 
city = missing_df_2["City"]
state = missing_df_2["State"]

geo_columns_1 = pd.concat([county, city, state], axis=1)
geo_columns_1.head()

,County,City,State
0,Maricopa,Goodyear,AZ
1,Santa Clara,Morgan Hill,CA
2,Polk,Mulberry,FL
3,Polk,Winter Haven,FL
4,Palm Beach,West Palm Beach,FL


In [128]:
geo_columns_1.describe()

,County,City,State
count,24980,25044,25060
unique,1402,5888,55
top,Los Angeles,Houston,CA
freq,939,453,3561


In [129]:
geo_columns_1.isnull().sum()

County    80
City      16
State      0
dtype: int64

In [130]:
for idx, i in enumerate(geo_columns_1["County"]): 
    if pd.isna(i) or i == "" or pd.isnull(i): 
        geo_columns_1.loc[idx, "County"] = "Unknown" 

for idx, i in enumerate(geo_columns_1["City"]): 
    if pd.isna(i) or i == "" or pd.isnull(i): 
        geo_columns_1.loc[idx, "City"] = "Unknown" 

In [131]:
geo_columns_1.isnull().sum()

County    0
City      0
State     0
dtype: int64

In [132]:
geo_columns_1.describe()

,County,City,State
count,25060,25060,25060
unique,1403,5888,55
top,Los Angeles,Houston,CA
freq,939,453,3561


In [133]:
county_1 = pd.get_dummies(geo_columns_1["County"], prefix="County")
city_1 = pd.get_dummies(geo_columns_1["City"], prefix="City") 
state_1 = pd.get_dummies(geo_columns_1["State"], prefix="State")

In [134]:
county_1.head()

,County_Abbeville,County_Acadia,County_Accomack,County_Ada,County_Adair,County_Adams,County_Addison,County_Aiken,County_Aitkin,County_Alachua,...,County_Yell,County_Yellowstone,County_Yolo,County_York,County_Young,County_Yuba,County_Yukon–Koyukuk,County_Yuma,County_Zapata,County_Zavala
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [135]:
city_1.head()

,City_Abbeville,City_Aberdeen,City_Abilene,City_Abingdon,City_Abington,City_Abita Springs,City_Accokeek,City_Ackerman,City_Acme,City_Acworth,...,City_Zephyrhills,City_Zig Zag,City_Zion,City_Zuni,City_knox county,City_mexico,City_queens,City_staten island,City_township,City_unknown
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [136]:
state_1.head()

,State_AK,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,...,State_TN,State_TX,State_UT,State_VA,State_VI,State_VT,State_WA,State_WI,State_WV,State_WY
0,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [137]:
final_db = pd.concat([final_db, county_1, city_1, state_1], axis=1)
final_db.head()

,Last Seen,Age,Gender_Female,Gender_Male,Gender_Other,American Indian / Alaska Native,Asian,Black / African American,Hispanic / Latino,Mix / Other / Unknown,...,State_TN,State_TX,State_UT,State_VA,State_VI,State_VT,State_WA,State_WI,State_WV,State_WY
0,-1.190250,-1.065312,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,-1.184895,-0.949558,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,-1.184895,0.034352,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,-1.184895,-1.065312,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,-1.184895,2.523065,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
### State Coords 

In [138]:
final_db["Cord_Long"] = 0
final_db["Cord_Lat"] = 0

state_coords = {
    "AZ": [34.0489, 111.0937], 
    "CA": [36.7783, 119.4179], 
    "FL": [27.6648, 81.5158], 
    "AR": [35.2010, 91.8318], 
    "CT": [41.6032, 73.0877], 
    "AL": [32.3182, 86.9023], 
    "CO": [39.5501, 105.7821], 
    "AK": [63.5888, 154.4931], 
    "DE": [38.9108, 75.5277], 
    "DC": [38.9072, 77.0369], 
    "MS": [32.3547, 89.3985], 

    "MI": [44.3148, 85.6024], 
    "OK": [35.0078, 97.0929], 
    "NC": [35.7596, 79.0193], 
    "IL": [40.6331, 89.3985], 
    "PA": [41.2033, 77.1945], 
    "HI": [19.8987, 155.6659], 
    "KY": [37.8393, 84.2700], 
    "SD": [43.9695, 99.9018], 
    "MD": [39.0458,  76.6413], 
    "MT": [46.8797, 110.3626], 
    "GA": [32.1574, 82.9071], 

    "NV": [38.8026, 116.4194], 
    "NJ": [40.0583, 74.4057], 
    "NY": [40.7128, 74.0060], 
    "NM": [34.9727, 105.0324], 
    "MO": [37.9643, 91.8318], 
    "OR": [43.8041, 120.5542], 
    "ND": [47.5515, 101.0020], 
    "LA": [30.5191, 91.5209], 
    "SC": [33.8361, 81.1637], 
    "MA": [42.4072, 71.3824], 
    "NE": [41.4925, 99.9018], 

    'IN': [40.5512, 85.6024], 
    'OH': [40.4173, 82.9071],
    'MN': [46.7296, 94.6859], 
    'KS': [39.0119, 98.4842], 
    'NH': [43.1939, 71.5724], 
    'ME': [45.2538, 69.4455],
    'IA': [41.8780, 93.0977], 
    'ID': [44.0682, 114.7420], 
    'RI': [41.5801, 71.4774], 
    'PR': [18.2208, 66.5901],
    'GU': [13.4443, 144.7937], 

    'MP': [15.0979, 145.6739], 
    'TX': [31.9686, 99.9018], 
    'TN': [35.5175, 86.5804], 
    'UT': [40.7607, 111.8939], 
    'VA': [37.4316, 78.6569], 
    'WI': [43.7844, 88.7879], 
    'WA': [47.7511, 120.7401], 
    'WY': [43.0760, 107.2903], 
    'WV': [38.5976, 80.4549], 
    'VT': [44.5588, 72.5778], 
    'VI': [18.3358, 64.8963], 
}

In [139]:
state.unique()

array(['AZ', 'CA', 'FL', 'AR', 'CT', 'AL', 'CO', 'AK', 'DE', 'DC', 'MS',
       'MI', 'OK', 'NC', 'IL', 'PA', 'HI', 'KY', 'SD', 'MD', 'MT', 'GA',
       'NV', 'NJ', 'NY', 'NM', 'MO', 'OR', 'ND', 'LA', 'SC', 'MA', 'NE',
       'IN', 'OH', 'MN', 'KS', 'NH', 'ME', 'IA', 'ID', 'RI', 'PR', 'GU',
       'MP', 'TX', 'TN', 'UT', 'VA', 'WI', 'WA', 'WY', 'WV', 'VT', 'VI'],
      dtype=object)

In [140]:
len(state_coords)

55

In [141]:
len(state.unique())

55

In [142]:
stateCoords = pd.DataFrame(missing_df_2["State"])
stateCoords.head()

,State
0,AZ
1,CA
2,FL
3,FL
4,FL


In [143]:
stateCoords["Lat"] = 0.0
stateCoords["Long"] = 0.0
stateCoords.head()

,State,Lat,Long
0,AZ,0.0,0.0
1,CA,0.0,0.0
2,FL,0.0,0.0
3,FL,0.0,0.0
4,FL,0.0,0.0


In [144]:
for idx, i in enumerate(stateCoords["State"]): 
    stateCoords.loc[idx, "Lat"] = state_coords[i][0]
    stateCoords.loc[idx, "Long"] = state_coords[i][1] 

In [145]:
stateCoords.head()

,State,Lat,Long
0,AZ,34.0489,111.0937
1,CA,36.7783,119.4179
2,FL,27.6648,81.5158
3,FL,27.6648,81.5158
4,FL,27.6648,81.5158


In [146]:
final_db.head()

,Last Seen,Age,Gender_Female,Gender_Male,Gender_Other,American Indian / Alaska Native,Asian,Black / African American,Hispanic / Latino,Mix / Other / Unknown,...,State_UT,State_VA,State_VI,State_VT,State_WA,State_WI,State_WV,State_WY,Cord_Long,Cord_Lat
0,-1.190250,-1.065312,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,0
1,-1.184895,-0.949558,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,0,0
2,-1.184895,0.034352,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,0
3,-1.184895,-1.065312,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,0
4,-1.184895,2.523065,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,0


In [147]:
final_db["Cord_Lat"] = stateCoords["Lat"]
final_db["Cord_Long"] = stateCoords["Long"]

In [148]:
final_db.head()

,Last Seen,Age,Gender_Female,Gender_Male,Gender_Other,American Indian / Alaska Native,Asian,Black / African American,Hispanic / Latino,Mix / Other / Unknown,...,State_UT,State_VA,State_VI,State_VT,State_WA,State_WI,State_WV,State_WY,Cord_Long,Cord_Lat
0,-1.190250,-1.065312,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,111.0937,34.0489
1,-1.184895,-0.949558,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,119.4179,36.7783
2,-1.184895,0.034352,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,81.5158,27.6648
3,-1.184895,-1.065312,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,81.5158,27.6648
4,-1.184895,2.523065,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,81.5158,27.6648
